## Model 1 (First method)

The Inception module is a type of convolutional neural network architecture that was first introduced in the GoogleNet paper. It is designed to improve the efficiency of feature extraction by using parallel branches of convolutions with different filter sizes. This allows the network to extract a wider range of features from the input image.

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dropout, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Define the Inception module
def inception_module(x):
    branch1 = Conv2D(filters=32, kernel_size=(1, 1), activation='relu', strides=(1, 1), padding='same')(x)

    branch2 = Conv2D(filters=32, kernel_size=(1, 1), activation='relu', strides=(1, 1), padding='same')(x)
    branch2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=(1, 1), padding='same')(branch2)

    branch3 = Conv2D(filters=64, kernel_size=(1, 1), activation='relu', strides=(1, 1), padding='same')(x)
    branch3 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=(1, 1), padding='same')(branch3)

    branch4 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(filters=32, kernel_size=(1, 1), activation='relu', strides=(1, 1), padding='same')(branch4)

    concatenated = tf.concat([branch1, branch2, branch3, branch4], axis=-1)
    return concatenated

# Define the convolutional neural network model
inputs = Input(shape=(32, 32, 3))

x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=(1, 1), padding='same')(inputs)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

x = inception_module(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
outputs = Dense(units=10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, zoom_range=0.2)

# Train the model
model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_test, y_test))


170498071/170498071 [==============================] - 3s 0us/step
Epoch 1/10
1563/1563 [==============================] - 48s 22ms/step - loss: 1.3319 - accuracy: 0.5249 - val_loss: 1.1410 - val_accuracy: 0.5905
Epoch 2/10
1563/1563 [==============================] - 38s 24ms/step - loss: 0.9839 - accuracy: 0.6531 - val_loss: 0.9110 - val_accuracy: 0.6854
Epoch 3/10
1563/1563 [==============================] - 38s 24ms/step - loss: 0.8472 - accuracy: 0.7037 - val_loss: 0.8792 - val_accuracy: 0.6959
Epoch 4/10
1563/1563 [==============================] - 42s 27ms/step - loss: 0.7582 - accuracy: 0.7340 - val_loss: 0.8489 - val_accuracy: 0.7149
Epoch 5/10
1563/1563 [==============================] - 37s 24ms/step - loss: 0.6844 - accuracy: 0.7602 - val_loss: 0.7623 - val_accuracy: 0.7403
Epoch 6/10
1563/1563 [==============================] - 37s 24ms/step - loss: 0.6222 - accuracy: 0.7807 - val_loss: 0.7747 - val_accuracy: 0.7432
Epoch 7/10
1563/1563 [==============================] - 3

Inception module: The Inception module is a convolutional neural network architecture that combines filters of different sizes in a single layer. This allows the network to capture a wider range of spatial and frequency information from the input data.

Stride parameter: The stride parameter in a convolutional layer determines how many pixels the filter moves across the input feature map after each convolution operation. A larger stride parameter will reduce the spatial dimensions of the output feature map.

Convolutional layers: Convolutional layers are the basic building blocks of convolutional neural networks. They apply filters to the input data to extract features. The key features of convolutional layers are their ability to extract local patterns and their ability to share weights across the input data.

Regularization and data augmentation: Regularization techniques, such as dropout, can help to prevent overfitting. Data augmentation techniques, such as horizontal and vertical flipping, can be used to increase the size of the training dataset.

## Model 2 (Second method)

## 1. Import Libraries and Load CIFAR-10 Data:


In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()


## 2. Preprocess Data:


In [4]:
# Normalize pixel values to range [0, 1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# One-hot encode labels
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)


## 3. Build the Inception Module:
The Inception module comprises parallel convolutional branches of different kernel sizes, followed by concatenation.

In [5]:
def inception_module(x):
    branch_1x1 = Conv2D(16, (1, 1), padding='same', activation='relu')(x)

    branch_3x3 = Conv2D(16, (1, 1), padding='same', activation='relu')(x)
    branch_3x3 = Conv2D(32, (3, 3), padding='same', activation='relu')(branch_3x3)

    branch_5x5 = Conv2D(16, (1, 1), padding='same', activation='relu')(x)
    branch_5x5 = Conv2D(32, (5, 5), padding='same', activation='relu')(branch_5x5)

    branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = Conv2D(16, (1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch_1x1, branch_3x3, branch_5x5, branch_pool], axis=-1)
    return output


## 4. Build the CNN with Inception Module:


In [6]:
input_layer = Input(shape=(32, 32, 3))

conv1 = Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_layer)
conv2 = Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(conv1)

inception = inception_module(conv2)

flatten = Flatten()(inception)
dense = Dense(128, activation='relu')(flatten)
output = Dense(10, activation='softmax')(dense)

model = Model(inputs=input_layer, outputs=output)


## 5. Compile and Train the Model:


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=20, batch_size=128, validation_data=(test_images, test_labels))


Epoch 1/20
391/391 [==============================] - 9s 16ms/step - loss: 1.4414 - accuracy: 0.4858 - val_loss: 1.1379 - val_accuracy: 0.6011
Epoch 2/20
391/391 [==============================] - 5s 13ms/step - loss: 0.9961 - accuracy: 0.6539 - val_loss: 0.9997 - val_accuracy: 0.6511
Epoch 3/20
391/391 [==============================] - 5s 13ms/step - loss: 0.8108 - accuracy: 0.7189 - val_loss: 0.8849 - val_accuracy: 0.6970
Epoch 4/20
391/391 [==============================] - 5s 14ms/step - loss: 0.6700 - accuracy: 0.7686 - val_loss: 0.8897 - val_accuracy: 0.6967
Epoch 5/20
391/391 [==============================] - 5s 14ms/step - loss: 0.5236 - accuracy: 0.8187 - val_loss: 0.9291 - val_accuracy: 0.7012
Epoch 6/20
391/391 [==============================] - 5s 13ms/step - loss: 0.3858 - accuracy: 0.8660 - val_loss: 1.0452 - val_accuracy: 0.6891
Epoch 7/20
391/391 [==============================] - 5s 14ms/step - loss: 0.2640 - accuracy: 0.9094 - val_loss: 1.1961 - val_accuracy: 0.6902

## Evaluate the model

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 4ms/step - loss: 2.5790 - accuracy: 0.6694
Test accuracy: 0.6693999767303467


## Key Points:
### Inception Module:
#### Purpose:
 The Inception module facilitates capturing features at multiple scales by using parallel convolutions of different kernel sizes (1x1, 3x3, 5x5) and pooling, allowing the network to learn diverse features and increase representational power.
### Stride Parameter in Convolutional Layers:
####Effect on Spatial Dimensions:
 A stride parameter > 1 reduces the spatial dimensions of feature maps. A stride of 2 reduces the spatial dimensions by a factor of 2, affecting the output size of the layer.
### Importance of Convolutional Layers:
#### Feature Extraction:
 Convolutional layers perform feature extraction by applying learnable filters to input data. Different filters capture different features (edges, textures) and play a crucial role in learning hierarchical representations.

## Model 3 (Third method)
 We can use various techniques like data augmentation, dropout, pooling, and more to improve the model's accuracy.

### First, let's import the necessary libraries and load the CIFAR-10 dataset:



In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Load and preprocess the CIFAR-10 dataset:



In [4]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoded vectors
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

### Next, let's define the Inception module:



In [5]:
def inception_module(prev_layer, filters):
    conv1x1 = Conv2D(filters=filters[0], kernel_size=(1, 1), activation='relu', padding='same')(prev_layer)

    conv3x3 = Conv2D(filters=filters[1], kernel_size=(3, 3), activation='relu', padding='same')(prev_layer)

    conv5x5 = Conv2D(filters=filters[2], kernel_size=(5, 5), activation='relu', padding='same')(prev_layer)

    maxpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(prev_layer)
    pool_conv = Conv2D(filters=filters[3], kernel_size=(1, 1), activation='relu', padding='same')(maxpool)

    # Concatenate the outputs of different convolutions
    output = concatenate([conv1x1, conv3x3, conv5x5, pool_conv], axis=-1)
    return output

### Now, let's build the CNN model with Inception module and other layers:


In [6]:
input_layer = Input(shape=(32, 32, 3))

# First Convolutional layer
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
conv1 = MaxPooling2D(pool_size=(2, 2))(conv1)

# Apply Inception module
inception1 = inception_module(conv1, [32, 32, 32, 32])

# Additional Convolutional layers
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(inception1)
conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Flatten the output for Dense layers
flatten = Flatten()(conv2)

# Dense layers
dense1 = Dense(256, activation='relu')(flatten)
dense1 = Dropout(0.5)(dense1)

output_layer = Dense(num_classes, activation='softmax')(dense1)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

### Compile the model and define the training parameters:



In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)

# Train the model
batch_size = 64
epochs = 30

history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))


Epoch 1/30
781/781 [==============================] - 40s 46ms/step - loss: 1.6481 - accuracy: 0.4003 - val_loss: 1.3300 - val_accuracy: 0.5313
Epoch 2/30
781/781 [==============================] - 35s 45ms/step - loss: 1.3183 - accuracy: 0.5278 - val_loss: 1.0763 - val_accuracy: 0.6199
Epoch 3/30
781/781 [==============================] - 33s 42ms/step - loss: 1.1946 - accuracy: 0.5782 - val_loss: 0.9964 - val_accuracy: 0.6439
Epoch 4/30
781/781 [==============================] - 33s 43ms/step - loss: 1.1061 - accuracy: 0.6102 - val_loss: 0.8739 - val_accuracy: 0.6939
Epoch 5/30
781/781 [==============================] - 32s 41ms/step - loss: 1.0430 - accuracy: 0.6336 - val_loss: 0.9179 - val_accuracy: 0.6751
Epoch 6/30
781/781 [==============================] - 35s 45ms/step - loss: 0.9932 - accuracy: 0.6515 - val_loss: 0.9343 - val_accuracy: 0.6847
Epoch 7/30
781/781 [==============================] - 33s 43ms/step - loss: 0.9622 - accuracy: 0.6649 - val_loss: 0.8114 - val_accuracy:

### Evaluate the model:



In [8]:
# Evaluate the model on test set
_, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 4ms/step - loss: 0.6225 - accuracy: 0.7939
Test accuracy: 79.39%


## Model 4 (Forth method)
### add L2 regularization to the Convolutional layers and Dense layers to see the results

In [9]:
from tensorflow.keras.regularizers import l2

# Define the regularization parameter
regularization_rate = 0.001  # You can adjust this value

input_layer = Input(shape=(32, 32, 3))

# First Convolutional layer with L2 regularization
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(regularization_rate))(input_layer)
conv1 = MaxPooling2D(pool_size=(2, 2))(conv1)

# Apply Inception module
inception1 = inception_module(conv1, [32, 32, 32, 32])

# Additional Convolutional layers with L2 regularization
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(regularization_rate))(inception1)
conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Flatten the output for Dense layers
flatten = Flatten()(conv2)

# Dense layers with L2 regularization
dense1 = Dense(256, activation='relu', kernel_regularizer=l2(regularization_rate))(flatten)
dense1 = Dropout(0.5)(dense1)

output_layer = Dense(num_classes, activation='softmax')(dense1)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

### Compile the model and define the training parameters:



In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)

# Train the model
batch_size = 64
epochs = 30

history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

Epoch 1/30
781/781 [==============================] - 36s 43ms/step - loss: 1.7983 - accuracy: 0.3807 - val_loss: 1.5050 - val_accuracy: 0.4913
Epoch 2/30
781/781 [==============================] - 32s 40ms/step - loss: 1.5302 - accuracy: 0.4951 - val_loss: 1.3507 - val_accuracy: 0.5543
Epoch 3/30
781/781 [==============================] - 31s 40ms/step - loss: 1.4479 - accuracy: 0.5315 - val_loss: 1.2351 - val_accuracy: 0.6027
Epoch 4/30
781/781 [==============================] - 31s 40ms/step - loss: 1.3887 - accuracy: 0.5552 - val_loss: 1.1894 - val_accuracy: 0.6319
Epoch 5/30
781/781 [==============================] - 31s 40ms/step - loss: 1.3475 - accuracy: 0.5751 - val_loss: 1.2259 - val_accuracy: 0.6231
Epoch 6/30
781/781 [==============================] - 34s 44ms/step - loss: 1.3139 - accuracy: 0.5930 - val_loss: 1.1979 - val_accuracy: 0.6355
Epoch 7/30
781/781 [==============================] - 31s 40ms/step - loss: 1.2879 - accuracy: 0.6030 - val_loss: 1.1051 - val_accuracy:

### Evaluate the model:



In [11]:
# Evaluate the model on test set
_, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 4ms/step - loss: 0.9490 - accuracy: 0.7409
Test accuracy: 74.09%


### This code creates a CNN model with an Inception module and utilizes data augmentation, dropout, and other techniques to achieve classification on the CIFAR-10 dataset.